In [2]:
import streamlit_functions as sf
import importlib as imp
import geopandas as gpd
import matplotlib.pyplot as plt
import functions as f
# debugging
import importlib as imp
imp.reload(f)

# graph
import networkx as nx

<module 'functions' from 'd:\\ITU\\Geospatial Data Science\\PROJECT\\functions.py'>

In [3]:
bikelane_all = gpd.read_parquet('dataset/raw_unprocessed/bikelane_dk_crs_25832.parquet')

In [44]:
G_edges = nx.Graph()

for row in bikelane_all.itertuples():
    # Access the MultiIndex values (u, v, key) from the Index
    u, v, _ = row.Index

    # Add nodes
    G_edges.add_node(u)
    G_edges.add_node(v)

    # Add edge
    G_edges.add_edge(u, v)


In [45]:
# Write to GraphML
nx.write_graphml(G_edges, 'graph.graphml')

In [6]:
coordinates1 = [(669963.1858043118, 6156595.619894134), (666199.299537813, 6155152.20113962)] # somewhere
coordinates2 = [(719420.4753819183, 6177679.685542446), (720676.1469959107, 6176793.722376416)] # in Copenhagen
coordinates3 = [(716235.4790146917, 6208167.005935997), (693675.6526506029, 6101757.298274318)] # big distance

In [72]:
bikelane_all.total_bounds

array([ 441698.70518066, 6072912.09409883,  728400.29103142,
       6402147.68476988])

### Without indexing the below code is super slow since it loops through the whole database to find the closest bikelane

In [46]:
import geopandas as gpd
import networkx as nx
from shapely.geometry import Point, LineString
from shapely.ops import nearest_points

# Assuming 'bikelane_all' is your GeoDataFrame of bike lanes
# and 'coordinates2' are your start and end points
coordinates2 = [(719420.4753819183, 6177679.685542446), (720676.1469959107, 6176793.722376416)] # in Copenhagen
# Convert points to Shapely Points
start_point = Point(coordinates2[0])
end_point = Point(coordinates2[1])

# H6 index? 

# Function to find the closest LineString to a point
print("findd closest linestring start")

# bikelane_all should have an extra H6 index to loop through it 

def find_closest_linestring(point, gdf):
    min_dist = float('inf')
    closest_line = None
    for line in gdf.geometry:
        dist = point.distance(line)
        if dist < min_dist:
            min_dist = dist
            closest_line = line
    return closest_line

# Find closest LineStrings to start and end points
closest_line_start = find_closest_linestring(start_point, bikelane_all)
closest_line_end = find_closest_linestring(end_point, bikelane_all)
print("closest_lines are:")
print(closest_line_start)
print(closest_line_end)

print("findd closest linestring end")
# Find closest nodes to start and end points
start_node, _ = nearest_points(start_point, closest_line_start)
end_node, _ = nearest_points(end_point, closest_line_end)
print("nearest_points end")
# Convert bike lanes network to a graph

"""G = nx.Graph()
for index, row in bikelane_all.iterrows():
    p1, p2 = row['geometry'].boundary
    G.add_edge(p1.coords[0], p2.coords[0], weight=row['geometry'].length)

# Find shortest path
path = nx.shortest_path(G, source=start_node.coords[0], target=end_node.coords[0], weight='weight')

# Convert path to LineString
path_linestring = LineString(path)

# Convert LineString to GeoDataFrame for visualization
path_gdf = gpd.GeoDataFrame(geometry=[path_linestring], crs=bikelane_all.crs)"""

findd closest linestring start
closest_lines are:
LINESTRING (719397.7148862425 6177829.284857129, 719405.0846778937 6177801.8819245845, 719421.6396323182 6177740.2992502665, 719423.9498579694 6177731.696480225, 719435.0308848498 6177689.015741999, 719444.0791641185 6177654.18242757, 719440.0535103269 6177646.34183948)
LINESTRING (720712.8564264507 6176820.9603884565, 720641.195113393 6176788.851636762)
findd closest linestring end
nearest_points end


"G = nx.Graph()\nfor index, row in bikelane_all.iterrows():\n    p1, p2 = row['geometry'].boundary\n    G.add_edge(p1.coords[0], p2.coords[0], weight=row['geometry'].length)\n\n# Find shortest path\npath = nx.shortest_path(G, source=start_node.coords[0], target=end_node.coords[0], weight='weight')\n\n# Convert path to LineString\npath_linestring = LineString(path)\n\n# Convert LineString to GeoDataFrame for visualization\npath_gdf = gpd.GeoDataFrame(geometry=[path_linestring], crs=bikelane_all.crs)"

### H3 indexing

In [48]:
import h3
import geopandas as gpd
from shapely.geometry import Point, LineString
from shapely.ops import nearest_points

# Assuming 'bikelane_all' is your GeoDataFrame of bike lanes
# and 'coordinates2' are your start and end points in Copenhagen
coordinates2 = [(669963.1858043118, 6156595.619894134), (666199.299537813, 6155152.20113962)] # somewhere

# Convert points to Shapely Points
start_point = Point(coordinates2[0])
end_point = Point(coordinates2[1])

# Choose an appropriate H3 resolution
h3_resolution = 8  # Example resolution, adjust based on your needs

# Function to assign H3 index to each linestring
def assign_h3_index_to_linestring(gdf, resolution):
    def get_h3_index_of_linestring(line):
        # Use the centroid of the line for indexing
        return h3.geo_to_h3(line.centroid.y, line.centroid.x, resolution)
    
    gdf['h3_index'] = gdf.geometry.apply(get_h3_index_of_linestring)
    return gdf

# Assign H3 index to each linestring in the GeoDataFrame
print("assign_h3_index_to_linestring")
bikelane_all = assign_h3_index_to_linestring(bikelane_all, h3_resolution)
print("assign_h3_index_to_linestring end")

# Function to find the closest LineString to a point, considering H3 index
def find_closest_linestring(point, gdf):
    point_h3_index = h3.geo_to_h3(point.y, point.x, h3_resolution)
    # Filter linestrings by H3 index
    filtered_gdf = gdf[gdf['h3_index'] == point_h3_index]
    
    min_dist = float('inf')
    closest_line = None
    for line in filtered_gdf.geometry:
        dist = point.distance(line)
        if dist < min_dist:
            min_dist = dist
            closest_line = line
    return closest_line

# Find closest LineStrings to start and end points
print("find_closest_linestring")
closest_line_start = find_closest_linestring(start_point, bikelane_all)
closest_line_end = find_closest_linestring(end_point, bikelane_all)
print("find_closest_linestring end")

# Find closest nodes to start and end points
start_node, _ = nearest_points(start_point, closest_line_start)
end_node, _ = nearest_points(end_point, closest_line_end)

assign_h3_index_to_linestring
assign_h3_index_to_linestring end
find_closest_linestring
find_closest_linestring end


ValueError: The second input geometry is empty

In [95]:
def swap_coordinates(coord):
    return coord.x, coord.y # Swap longitude and latitude

def find_closest_linestring(point, gdf):
    point = swap_coordinates(point)
    print(point[0], point[1])
    point_h3_index = h3.geo_to_h3(point[0], point[1], h3_resolution)
    print("point_h3_index")
    print(point_h3_index)
    # Filter linestrings by H3 index
    filtered_gdf = gdf[gdf['h3_index'] == point_h3_index]
    print("filtered_gdf")
    print(filtered_gdf)
    
    min_dist = float('inf')
    closest_line = None
    for line in filtered_gdf.geometry:
        dist = point.distance(line)
        if dist < min_dist:
            min_dist = dist
            closest_line = line
    return closest_line

coordinates2 = [(704132.978 , 6138611.037), (704094.801 , 6138413.968)] # somewhere
print(bikelane_all.total_bounds)
# Convert points to Shapely Points
start_point = Point(coordinates2[0])
end_point = Point(coordinates2[1])

closest_line_start = find_closest_linestring(end_point, bikelane_all)

[ 441698.70518066 6072912.09409883  728400.29103142 6402147.68476988]
704094.801 6138413.968
point_h3_index
88e14655c3fffff
filtered_gdf
Empty GeoDataFrame
Columns: [geometry, h3_index]
Index: []


In [97]:
bikelane_all

geometry  \
u           v           key                                                      
676544      443246370   0    LINESTRING (704132.978 6138611.037, 704094.801...   
            676545      0    LINESTRING (704132.978 6138611.037, 704172.669...   
            443245726   0    LINESTRING (704132.978 6138611.037, 704146.202...   
726713      443257088   0    LINESTRING (704670.169 6138065.010, 704522.145...   
            1478811288  0    LINESTRING (704670.169 6138065.010, 704680.236...   
...                                                                        ...   
11755267988 11755267989 0    LINESTRING (572878.566 6156776.331, 572822.397...   
            1722039621  0    LINESTRING (572878.566 6156776.331, 572879.678...   
11755267989 11755267988 0    LINESTRING (572822.397 6156764.722, 572878.566...   
11755267999 1316680106  0    LINESTRING (570752.832 6159487.934, 570774.534...   
11755268000 1316680106  0    LINESTRING (570772.965 6159703.835, 570745.941...   

                                    h3_index  
u           v           key                   
676544      443246370   0    884b5a151dfffff  
            676545      0    88ed25494dfffff  
            443245726   0    88c0305031fffff  
726713      443257088   0    880b16b963fffff  
            1478811288  0    884e83cb2dfffff  
...                                      ...  
11755267988 11755267989 0    8825322409fffff  
            1722039621  0    88736679adfffff  
11755267989 11755267988 0    8825322409fffff  
11755267999 1316680106  0    889f507911fffff  
11755268000 1316680106  0    88004ec75bfffff  

[3257915 rows x 2 columns]

In [104]:
def swap_coordinates(coord):
    return coord.y, coord.x # Swap longitude and latitude

coordinates2 = [(704132.978 , 6138611.037), (704094.801 , 6138413.968)] # somewhere
end_point = Point(coordinates2[0])
point = swap_coordinates(end_point)
print(point[0], point[1])
h3.geo_to_h3(point[0], point[1], h3_resolution)

6138611.037 704132.978


'88ec2e7265fffff'

In [106]:
import h3
import folium

# Example H3 index
h3_index = '88ec2e7265fffff'  # Replace with your H3 index

# Convert the H3 index to a GeoJSON polygon
h3_boundary = h3.h3_to_geo_boundary(h3_index, geo_json=True)

# Create a Folium map at a location centered on the H3 index
map = folium.Map(location=[h3_boundary[0][0], h3_boundary[0][1]], zoom_start=12)

# Create a polygon feature and add it to the map
folium.Polygon(locations=h3_boundary, color='blue', fill=True).add_to(map)

# Display the map
map

In [105]:
def swap_coordinates(coord):
    return coord.x, coord.y # Swap longitude and latitude

coordinates2 = [(704132.978 , 6138611.037), (704094.801 , 6138413.968)] # somewhere
end_point = Point(coordinates2[0])
point = swap_coordinates(end_point)
print(point[0], point[1])
h3.geo_to_h3(point[0], point[1], h3_resolution)

704132.978 6138611.037


'88b0a245e1fffff'

In [107]:
import h3
import folium

# Example H3 index
h3_index = '88b0a245e1fffff'  # Replace with your H3 index

# Convert the H3 index to a GeoJSON polygon
h3_boundary = h3.h3_to_geo_boundary(h3_index, geo_json=True)

# Create a Folium map at a location centered on the H3 index
map = folium.Map(location=[h3_boundary[0][0], h3_boundary[0][1]], zoom_start=12)

# Create a polygon feature and add it to the map
folium.Polygon(locations=h3_boundary, color='blue', fill=True).add_to(map)

# Display the map
map

In [102]:
bikelane_all.head(1).geometry.values

<GeometryArray>
[<LINESTRING (704132.978 6138611.037, 704094.801 6138413.968)>]
Length: 1, dtype: geometry

In [108]:
bikelane_all.head(1).h3_index.values

array(['884b5a151dfffff'], dtype=object)

In [111]:
import h3
import folium
from shapely.geometry import Polygon

# Example H3 index
h3_index = '884b5a151dfffff'  # Replace with your H3 index

# Convert H3 index to boundary coordinates and then to a Shapely Polygon
h3_boundary = h3.h3_to_geo_boundary(h3_index, geo_json=True)
polygon = Polygon(h3_boundary)

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame([1], geometry=[polygon], crs=f.DENMARK_CRS)
gdf.explore()

AssertionError: The field 0 is not available in the data. Choose from: (0,).

In [113]:
gdf

,0,geometry
0,1,"POLYGON ((133.888 27.499, 133.887 27.495, 133...."


In [103]:
bikelane_all.head(1)

,,,geometry,h3_index
u,v,key,,
676544,443246370,0,"LINESTRING (704132.978 6138611.037, 704094.801...",884b5a151dfffff


In [77]:
bounds = bikelane_all.geometry.total_bounds
print(bounds)

from shapely.geometry import box
import geopandas as gpd

# Create a Polygon from bounds
bound_box = box(*bounds)

# Convert to a GeoDataFrame
gdf_bounds = gpd.GeoDataFrame(geometry=[bound_box], crs=bikelane_all.crs)

print(gdf_bounds)

# check these coordinates are inside the box 



[ 441698.70518066 6072912.09409883  728400.29103142 6402147.68476988]
                                            geometry
0  POLYGON ((728400.291 6072912.094, 728400.291 6...


In [84]:
gdf_bounds.explore()

In [78]:
from shapely.geometry import Point

# Convert coordinates to Point geometries
points = gpd.GeoSeries([Point(xy) for xy in coordinates2], crs=bikelane_all.crs)

# Check if the bounding box contains these points
contains_points = gdf_bounds.geometry.contains(points)

print(contains_points)

0     True
1    False
dtype: bool


C:\Users\Gergo Gyori\AppData\Local\Temp\ipykernel_8216\2082974034.py:7: UserWarning: The indices of the two GeoSeries are different.
  contains_points = gdf_bounds.geometry.contains(points)


In [79]:
coordinates2

[(669963.1858043118, 6156595.619894134), (666199.299537813, 6155152.20113962)]